### Agentic RAG

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool
from langchain_community.utilities import GoogleSerperAPIWrapper

In [2]:
@tool("GoogleSearch")
def search(query_string: str):
    """
    Useful to search for any kinds of information and
    when you need to search the internet for any kinds of information, use this tool.
    Prefer this tool when you search for long queries.
    Should not be used for Article search or Topic Search.
    """
    
    search = GoogleSerperAPIWrapper()
    
    return search.run(query_string)

In [4]:

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
wiki = WikipediaQueryRun(
    name="WikiepdiaSearch",
    description="Use this tool when you want to analyze for information on Wikipedia by Terms, Keywords or any Topics.",
    api_wrapper=api_wrapper)

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
).split_documents(docs)

vectordatabase = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordatabase.as_retriever()

In [7]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "search for information about langsmith. for any questions related to langsmith, you must use this tool"
)

retriever_tool.name

'langsmith_search'

In [8]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [9]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [10]:
tools = [arxiv, search, wiki, retriever_tool]
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=1000)),
 StructuredTool(name='GoogleSearch', description='Useful to search for any kinds of information and\nwhen you need to search the internet for any kinds of information, use this tool.\nPrefer this tool when you search for long queries.\nShould not be used for Article search or Topic Search.', args_schema=<class 'langchain_core.utils.pydantic.GoogleSearch'>, func=<function search at 0x000001B26D05B600>),
 WikipediaQueryRun(name='WikiepdiaSearch', description='Use this tool when you want to analyze for information on Wikipedia by Terms, Keywords or any Topics.', api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\000 - SONY A

In [11]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

In [12]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o",
    max_tokens=2000,
    temperature=0.1,
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

In [13]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\000 - SONY Agentic AI\demonstrations\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage

agent_executor = create_react_agent(
    llm,
    tools = tools
)

In [16]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="tell me about langsmith")
    ]
}):
    print(stream)
    print("***********")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_f5kHGofkzwF9bRmT1zKO7NJ0', 'function': {'arguments': '{"query":"langsmith"}', 'name': 'langsmith_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 255, 'total_tokens': 271, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': None, 'id': 'chatcmpl-BtXK5zXK9lc60cbeq7l7Rja9vnfeo', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a960417e-e543-49c0-8d52-d761dbb55a4f-0', tool_calls=[{'name': 'langsmith_search', 'args': {'query': 'langsmith'}, 'id': 'call_f5kHGofkzwF9bRmT1zKO7NJ0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 255, 'output_tokens': 16, 'total_tokens': 271,

In [17]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="whats the paper 2412.16446 talk about it?")
    ]
}):
    print(stream)
    print("***********")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Kq8s2SI08WW7gR1fOelFrLr1', 'function': {'arguments': '{"query":"2412.16446"}', 'name': 'arxiv'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 264, 'total_tokens': 282, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': None, 'id': 'chatcmpl-BtXKT5Fsor9pi6HDLOMtBCiFtXpLa', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7db40a4b-d2ef-4cb6-84bd-e3337d252e9c-0', tool_calls=[{'name': 'arxiv', 'args': {'query': '2412.16446'}, 'id': 'call_Kq8s2SI08WW7gR1fOelFrLr1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 264, 'output_tokens': 18, 'total_tokens': 282, 'input_token_detail

In [20]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="Indian Constitution")
    ]
}):
    print(stream)
    print("***********")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_T4DEdEIiuF5zUemY5rqj40eE', 'function': {'arguments': '{"query":"Indian Constitution"}', 'name': 'WikiepdiaSearch'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 252, 'total_tokens': 271, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': None, 'id': 'chatcmpl-BtXMD6QKEBOIN5zZHKTSK4fFwd9Gk', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--57f96da2-761c-4e39-a223-03589bd86fc6-0', tool_calls=[{'name': 'WikiepdiaSearch', 'args': {'query': 'Indian Constitution'}, 'id': 'call_T4DEdEIiuF5zUemY5rqj40eE', 'type': 'tool_call'}], usage_metadata={'input_tokens': 252, 'output_tokens': 19, 't

In [22]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="Who won the world cup in the Year 2025 in WTC?")
    ]
}):
    print(stream)
    print("***********")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6BIoSj9ra1f7QLTOnCJae64X', 'function': {'arguments': '{"query_string":"2025 World Test Championship winner"}', 'name': 'GoogleSearch'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 265, 'total_tokens': 285, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': None, 'id': 'chatcmpl-BtXNHaXXSoE4Q0Bw6TkdnmGOPUQmV', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a7ff6041-d867-45a0-abc0-5688aa24fada-0', tool_calls=[{'name': 'GoogleSearch', 'args': {'query_string': '2025 World Test Championship winner'}, 'id': 'call_6BIoSj9ra1f7QLTOnCJae64X', 'type': 'tool_call'}], usage_metadata={'inp